<a href="https://colab.research.google.com/github/FyodorPi/control-charts/blob/main/control_chart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install JPype1

In [ ]:
import pandas as pd
import numpy as np
import openpyxl
import os
import glob
import statistics as stat
import math
import jpype
# import asposecells
jpype.startJVM()
# from asposecells.api import Workbook as Workbook1
# import ezodf
from openpyxl import load_workbook, Workbook
from openpyxl.styles import Color, PatternFill, Font, Border, Alignment
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import ColorScaleRule, CellIsRule, FormulaRule
from openpyxl.chart import BarChart, LineChart, ScatterChart, Reference, Series
from openpyxl.worksheet.table import Table, TableStyleInfo
from pandas.io.excel import ExcelWriter
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# product = ('arm_kanat', 'Арматурный канат')
# product = ('prgon', 'Противоугон')
product = ('p_prov', 'Пружинная проволока')

In [ ]:
# os.chdir('..')
os.chdir(f"/content/drive/My Drive/1Статистика/Статистика/oracle_{product[0]}")

xl_files = glob.glob('*')

In [ ]:
combined = pd.DataFrame()
combined2 = pd.DataFrame()
combined3 = pd.DataFrame()

# Цикл по файлам
for xl_file in xl_files:
    xl_file_obj = pd.ExcelFile(xl_file)

    for sheet_name in xl_file_obj.sheet_names:
        results = 'Результаты'
        probes = 'Пробы'
        vhcontrol = 'Вх.контроль'

        if sheet_name.startswith(results):
            data = pd.read_excel(xl_file_obj,
                                # skiprows=15,
                                header=[0],
                                sheet_name=sheet_name)

            data = data.drop(labels = [0],axis = 0)
            data=data.rename(columns = {'Unnamed: 10':'Минимум_норм', 'Unnamed: 11':'Максимум_норм'})
            # print(data)
            data = data.astype({'Минимум': np.float64})
            data = data.astype({'Максимум': np.float64})
            data = data.dropna (subset=['Минимум'])
            data = data.dropna (subset=['Максимум'])
            data['mean'] = data[['Минимум', 'Максимум']].mean(axis= 1 )

            combined = pd.concat([combined, data], sort=False)
            # print(combined)

        if sheet_name.startswith(probes):
            data2 = pd.read_excel(xl_file_obj,
                                header=[0],
                                sheet_name=sheet_name)
            combined2 = pd.concat([combined2, data2], sort=False)
            combined2["Диаметр"] = combined2["Описание позиции"].str.extract('(\d+(?:,\d+)?\s)', expand=False)
            combined2['Диаметр'] = combined2['Диаметр'].str.replace(',', '.')
            # combined2['Диаметр'] = combined2['Диаметр'].str.replace(' ', '')
            combined2 = combined2.astype({'Диаметр': np.float64})
            combined2['Размер-марка-НД'] = combined2['Описание позиции'].str.replace('Проволока ', '')

        if sheet_name.startswith(vhcontrol):
            data3 = pd.read_excel(xl_file_obj,
                                header=[0],
                                sheet_name=sheet_name)
            data3 = data3.drop(labels = [0],axis = 0)
            combined3 = pd.concat([combined3, data3], sort=False)
            # print(combined3)

# elem = 'C'
elem = 'Mn'

combined4 = combined3.loc[combined3["Показатель"] == elem]
# combined5 = combined3.loc[combined3["Показатель"] == 'Врм_сопр']
combined4 = combined4.astype({'Данные СК поставщика': np.float64})
# combined5 = combined5.astype({'Данные СК поставщика': np.float64})
combined4 = combined4.groupby(['Плавка'])['Данные СК поставщика'].agg(["mean"])
combined4 = combined4.rename(columns={'mean': elem})
# combined4 = combined4.rename(columns={'mean': 'Ср. Mn'})
# combined5 = combined5.groupby(['Плавка'])['Данные СК поставщика'].agg(["mean"])
# combined5 = combined5.rename(columns={'mean': 'Ср. Врм_сопр'})

combine=pd.merge(combined, combined2, on=('Проба'), how ='left')
combine=pd.merge(combine, combined4, on=('Плавка'), how ='left')
# combine=pd.merge(combine, combined5, on=('Плавка'), how ='left')
# print(combine)

combine = pd.DataFrame(data=combine)

combine['Код проверки'] = combine['Код проверки'].str.replace('|', '-')
combine['Код проверки'] = combine['Код проверки'].str.replace('/', '-')
combine['Код проверки'] = combine['Код проверки'].str.replace('-М', '')
combine['Класс проверки (код)'] = combine['Класс проверки (код)'].str.replace('|', '-')
combine['Класс проверки (код)'] = combine['Класс проверки (код)'].str.replace('М-', '')

# Удаление строк с пустыми значениями в поле "Минимум"
combine = combine.dropna (subset=['Минимум'])

# print(combine)

In [ ]:
# os.chdir('..')
os.chdir(f"/content/drive/My Drive/1Статистика/Статистика/result_{product[0]}")

In [ ]:
# n_data=len(combine.index)
# n_kart = 0
# n_dev = 0
# d_m_nd = []
# Cp1 = []
# Cpk1 = []

for klass in ['МЕХСВ']:
# for klass in combine['Класс проверки (код)'].unique ():
    print(klass)
    # Выборка по классу свойств
    combin = combine.loc[combine["Класс проверки (код)"] == klass]

    # for charact in combin['Код проверки'].unique ():
    for charact in [
        'Врм_сопр',
        'Число_скручиваний',
        ]:
        # Выборка по свойству
        combi = combin.loc[combin["Код проверки"] == charact]

        # for d_marka_nd in combi['Размер-марка-НД'].unique ():
        for d_marka_nd in [
            # 'Б-2-1,7 ГОСТ 9389-75 (СПИК)',
            # 'Б-2-1,75 ГОСТ 9389-75 (СПИК)',
            'Б-2-1,8 ГОСТ 9389-75 (СПИК)',
            ]:
            # print(marka)
            # Выборка по марке стали
            comb = combi.loc[combi["Размер-марка-НД"] == d_marka_nd]
            # comb = combi.loc[combi["Марка-НД"] == 'Б-2-1,8 ГОСТ 9389-75 (СПИК)']

            # Указать writer библиотеки
            writer1=pd.ExcelWriter('Карты ' + str(d_marka_nd) + ' ' +
                    str(charact) + '.xlsx', engine='openpyxl')
            wb  = writer1.book

            # for diam in comb['Диаметр'].unique ():
            #     # print(diam)
            #     # Выборка по состоянию
                # com = comb.loc[comb["Диаметр"] == diam]
            for minimum in comb['Минимум_норм'].unique ():
                    # print(min)
                # Выборка по нижней технической границе
                    df1 = comb.loc[comb['Минимум_норм'] == minimum]
                    # print(df['Максимум'].unique ())
                    for maximum in df1['Максимум_норм'].unique ():
                        # print(max)
                # Выборка по верхней технической границе
                        df = df1.loc[df1['Максимум_норм'] == maximum]
                        pd.options.mode.chained_assignment = None
                        df['Дата создания пробы1'] = df['Дата создания пробы']
                        df['Дата создания пробы1'] = pd.to_datetime(df["Дата создания пробы1"],
                                                                        format="%d.%m.%Y %H:%M:%S")
                        df = df.sort_values('Дата создания пробы1')
                        df['Дата создания пробы1'] = pd.to_datetime(df['Дата создания пробы1']).dt.date
                        df['Дата создания пробы1']=pd.to_datetime(df['Дата создания пробы1'].astype(str))
                        df['Дата создания пробы1'] = df['Дата создания пробы1'].dt.strftime('%d.%m.%Y')
                        # pd.options.display.max_columns = None
                        df = df.loc[df["Описание марки"] == '65']
                        # print(df["Описание марки"])
                        # df = df.loc[(df[elem] < 0.46)]
                        # df = df.loc[(df[elem] >= 0.46) & (df[elem] < 0.5)]
                        # df = df.loc[(df[elem] >= 0.5)]
                        # df = df.loc[(df['Плавка'] == 450110)]
                        df = df.loc[(df['Плавка'] == 891364)]

                        # print(df['date'])
                        df['mean1'] = df['mean']. shift (1)
                        df['mr'] = pd.Series.abs(df['mean'] - df['mean1'])
                        # print(df['mr'])
                        n_strok=len(df.index )

                        ws = wb.create_sheet(str(minimum) +'-'+ str(maximum))

                       # Ширина ячейки
                        ws.column_dimensions['A'].width = 15
                        ws.column_dimensions['B'].width = 15
                        # Вставляем текст в ячейку
                        ws['A2']='Наименование продукции'
                        ws['B2']='Проволока'
                        ws['A3']='Марка-НД'
                        ws['B3'] = str(d_marka_nd)
                        ws['A4']='Диаметр'
                        # ws1['B4'] = str(diam)
                        ws['A5']='Характеристика'
                        ws['B5'] = charact
                        ws['A6']='LSL'
                        ws['B6']= minimum
                        ws['A7']='USL'
                        ws['B7']= maximum

                        Xsrsr = df['mean'].mean()
                        MRsr = df['mr'].mean()
                        if n_strok > 24:
                            list1 = df['Минимум']. tolist ()
                            # print(list1)
                            list2 = df['Максимум']. tolist ()
                            # print(list2)
                            list3 = list1 + list2
                            St = np.std(list3)
                            # print(St)
                            S = MRsr/1.128
                            ws['A8']='LCL,Xср'
                            ws['B8']= round(Xsrsr-2.66*MRsr, 2)
                            ws['A9']='UCL,Xср'
                            ws['B9']= round(Xsrsr+2.66*MRsr, 2)
                            ws['A10']='CL,Xср'
                            ws['B10']= round(Xsrsr, 2)
                            ws['A11']='UCL,MR'
                            ws['B11']= round(3.267*MRsr, 2)
                            ws['A12']='CL,MR'
                            ws['B12']= round(MRsr, 2)
                            minimum = float(minimum)
                            maximum = float(maximum)

                            ws['A13']='Ppk'
                            Ppk_sup = (maximum-Xsrsr)/(3*St)
                            Ppk_inf = (Xsrsr-minimum)/(3*St)
                            Ppk = min([Ppk_sup, Ppk_inf])
                            ws['B13']= round(Ppk, 2)

                            ws['A15'] = 'Cpk'
                            Cpk_sup = (maximum-Xsrsr)/(3*S)
                            Cpk_inf = (Xsrsr-minimum)/(3*S)
                            Cpk = min([Cpk_sup, Cpk_inf])
                            ws['B15'] = round(Cpk, 2)

                            ws['A14']='Pp'
                            ws['A16'] = 'Cp'
                            if minimum == 0 or maximum == 100:
                                ws['B14'] = 'не применимо'
                                ws['B16'] = 'не применимо'
                                Cp = '-'
                            else:
                                if maximum/minimum > 1.5:
                                    ws['B14'] = 'не применимо'
                                    ws['B16'] = 'не применимо'
                                    Cp = '-'
                                else:
                                    Pp = (maximum-minimum)/(6*St)
                                    Cp = (maximum-minimum)/(6*S)
                                    ws['B14'] = round(Pp, 2)
                                    ws['B16'] = round(Cp, 2)
                        else:
                            Cp = '-'
                            Cpk = '-'

                            def func(x):
                                return math.exp(-(x*x)/4)

                            def trianmethod(a, b, n):
                                result = 0
                                h = (b - a) / n
                                # print(h)
                                for j in range(n):
                                    result += func(a + h * (j + 0.5))
                                    # print(result)
                                result *= h
                                # print(result)
                                return result



                        for i in range(0,n_strok):
                            ws.cell(row=30,column=i+4).value = i+1
                            ws.cell(row=i+51,column=1).value = i+1
                            ws.cell(row=31,column=i+4).value = df['Дата создания пробы1'].iloc[i]
                            # ws1.cell(row=i+51,column=1).value = df['Дата создания пробы'].iloc[i]
                            # ws1.cell(row=32,column=i+4).value = df['Размер, мм'].iloc[i]
                            ws.cell(row=33,column=i+4).value = df['Плавка'].iloc[i]
                            ws.cell(row=34,column=i+4).value = df['Партия_x'].iloc[i]
                            ws.cell(row=35,column=i+4).value = df['Минимум'].iloc[i]
                            ws.cell(row=36,column=i+4).value = df['Максимум'].iloc[i]
                            ws.cell(row=37,column=i+4).value = df['mean'].iloc[i]
                            ws.cell(row=i+51,column=2).value = df['mean'].iloc[i]
                            ws.cell(row=38,column=i+4).value = df['mr'].iloc[i]
                            ws.cell(row=i+51,column=8).value = df['mr'].iloc[i]
                            # ws1.cell(row=39,column=i+4).value = df['Размер г/к пр, мм'].iloc[i]
                            ws.cell(row=39,column=i+4).value = df['Описание марки'].iloc[i]
                            ws.cell(row=40,column=i+4).value = df[elem].iloc[i]
                            # ws1.cell(row=40,column=i+4).value = df['Ср. Mn'].iloc[i]
                            # ws.cell(row=41,column=i+4).value = df['Ср. Врм_сопр'].iloc[i]

                            ws.cell(row=i+51,column=3).value = minimum
                            ws.cell(row=i+51,column=4).value = maximum
                            ws.cell(row=i+51,column=5).value = Xsrsr-2.66*MRsr
                            ws.cell(row=i+51,column=6).value = Xsrsr+2.66*MRsr
                            ws.cell(row=i+51,column=7).value = Xsrsr
                            ws.cell(row=i+51,column=9).value = 3.267*MRsr
                            ws.cell(row=i+51,column=10).value = MRsr

                        ws.cell(row=50,column=2).value = 'Значения X'
                        ws.cell(row=50,column=3).value = 'Нижняя техническая граница LSL,X'
                        ws.cell(row=50,column=4).value = 'Верхняя техническая граница USL,X'
                        ws.cell(row=50,column=5).value = 'Нижняя контрольная граница LCL,X'
                        ws.cell(row=50,column=6).value = 'Верхняя контрольная граница UCL,X'
                        ws.cell(row=50,column=7).value = 'Среднее значение CL,X'
                        ws.cell(row=50,column=8).value = 'Скользящий размах MR'
                        ws.cell(row=50,column=9).value = 'Верхняя контрольная граница UCL,MR'
                        ws.cell(row=50,column=10).value = 'Среднее значение CL,MR'


                        ws['B30']='№ подгруппы'
                        ws['B31']='Дата'
                        ws['B32']='Размер, мм'
                        ws['B33']='Плавка'
                        ws['B34']='Партия'
                        ws['B35']='Xmin'
                        ws['B36']='Xmax'
                        ws['B37']='Xср'
                        ws['B38']='MR'
                        ws['B39']='Марка стали'
                        # ws1['B40']='C,%'
                        ws['B40']= elem + ',%'
                        # ws['B41']='Врм.сопр.кат.'
                        ws.merge_cells('B30:C30')
                        ws.merge_cells('B31:C31')
                        ws.merge_cells('B32:C32')
                        ws.merge_cells('B33:C33')
                        ws.merge_cells('B34:C34')
                        ws.merge_cells('B35:C35')
                        ws.merge_cells('B36:C36')
                        ws.merge_cells('B37:C37')
                        ws.merge_cells('B38:C38')
                        ws.merge_cells('B39:C39')
                        ws.freeze_panes = 'D1'

                        # создаем объект столбчатой диаграммы
                        chart = LineChart()
                        # заголовок диаграммы
                        # chart1.title = "Линейная диаграмма"
                        # установим цветовую схему диаграммы
                        chart.style = 13
                            # подпись оси `y`
                        chart.y_axis.title = 'Карта средних значений'

                        # chart.x_axis.delete = True
                        # print(df)
                        if df.empty:
                            break
                        chart.y_axis.scaling.min = math.floor(10*((0.9*df['mean'].min()/10)))
                        chart.y_axis.scaling.max = math.ceil(10*((1.1*df['mean'].max()/10)))

                            # создаем объект столбчатой диаграммы
                        chart2 = LineChart()
                            # заголовок диаграммы
                            # chart1.title = "Линейная диаграмма"
                            # установим цветовую схему диаграммы
                        chart2.style = 13
                            # подпись оси `y`
                        chart2.y_axis.title = 'Карта скользящих размахов'

                        # chart2.x_axis.delete = True

                           # создаем объект столбчатой диаграммы
                        chart3 = ScatterChart()
                            # заголовок диаграммы
                            # chart1.title = "Линейная диаграмма"
                            # установим цветовую схему диаграммы
                        chart3.style = 13
                            # подпись оси `y`
                        # chart3.y_axis.title = 'Врем.сопр. от содержания С'
                        # chart3.y_axis.title = 'Врем.сопр. от содержания Mn'


                        # выбираем диапазоны значений для диаграммы
                        data = Reference(ws,
                                         min_col=2,
                                         max_col=7,
                                         min_row=51,
                                         max_row=n_strok+50)

                        data2 = Reference(ws,
                                         min_col=8,
                                         max_col=10,
                                         min_row=51,
                                         max_row=n_strok+50)

                        categories = Reference(ws,
                                         min_col=1,
                                         max_col=1,
                                         min_row=51,
                                         max_row=n_strok+50)

                        values = Reference(ws,
                                         min_col=2,
                                         max_col=2,
                                         min_row=51,
                                         max_row=n_strok+50)

                        xvalues3 = Reference(ws,
                                         min_col=4,
                                         max_col=n_strok+3,
                                         min_row=40,
                                         max_row=40)

                        series3 = Series(values, xvalues3, title_from_data=False)

                            # добавляем в диаграмму выбранный диапазон значений
                        chart.add_data(data, titles_from_data=True)
                        chart.set_categories(categories)
                        chart2.add_data(data2, titles_from_data=True)
                        chart2.set_categories(categories)

                        chart3.series.append(series3)
                        # chart4.series.append(series4)

                            # Style the lines
                        s1 = chart.series[0]
                        s2 = chart.series[1]
                        s3 = chart.series[2]
                        s4 = chart.series[3]
                        s4.graphicalProperties.line.dashStyle = "sysDot"
                        s5 = chart.series[4]
                        s5.graphicalProperties.line.dashStyle = "sysDot"
                        s6 = chart.series[5]
                        s6.graphicalProperties.line.dashStyle = "dashDot"
                        s1.marker.symbol = "triangle"
                        s1.marker.graphicalProperties.solidFill = "00AAAA" # Marker filling
                        s1.graphicalProperties.line.solidFill = "00AAAA"
                            # привязываем диаграмму к ячейке `E15`
                        ws.add_chart(chart, "C1")
                            # определяем размеры диаграммы в сантиметрах
                        chart.width = n_strok*1.69+4
                        chart.height = 7.5

                        #  Style the lines
                        s7 = chart2.series[0]
                        s8 = chart2.series[1]
                        s8.graphicalProperties.line.dashStyle = "sysDot"
                        s9 = chart2.series[2]
                        s9.graphicalProperties.line.dashStyle = "dashDot"
                        s7.marker.symbol = "triangle"
                        s7.marker.graphicalProperties.solidFill = "00AAAA" # Marker filling
                        s7.graphicalProperties.line.solidFill = "00AAAA"
                            # привязываем диаграмму к ячейке `E15`
                        ws.add_chart(chart2, "C15")
                            # определяем размеры диаграммы в сантиметрах
                        chart2.width = n_strok*1.69+4
                        chart2.height = 7.5

                        s10 = chart3.series[0]
                        s10.marker.symbol = "triangle"
                        s10.marker.graphicalProperties.solidFill = "FF0000" # Marker filling
                        s10.marker.graphicalProperties.line.solidFill = "FF0000" # Marker outline
                        s10.graphicalProperties.line.noFill = True  # hide lines
                            # привязываем диаграмму к ячейке `E15`
                        ws.add_chart(chart3, "L42")

            wb.save('Карты ' + str(d_marka_nd) + ' ' +
                    str(charact) + '.xlsx')
            # workbook1 = Workbook1('Карты ' + str(marka) + ' ' +
            #         str(charact) + '.xlsx')
            # workbook1.save('Карты ' + str(marka) + ' ' +
            #         str(charact) + '.ods')
            # workbook2 = ezodf.opendoc('Карты ' + str(marka) + ' ' +
            #         str(charact) + '.ods')
            # # list(workbook2.sheets.names())
            # del workbook2.sheets['Evaluation Warning']
            # workbook2.save()
            # path = 'table_Oracle/*.bak'
            # os.remove(path)

jpype.shutdownJVM()




МЕХСВ
